Primary Objective: complete all the steps below and return the risk analysis of your seven (7) stock portfolio
against the S&P500 (SPY), Russell 2000 (IWM), and the Dow Jones Industrial Average (DIA).

Requirements:
3. Create a correlation matrix showing the correlations between the equal-weighted portfolio, 3 ETFs, and your 7 stocks.
4. Output these two tables and the matrix into a PDF file. You must submit both your Python code file and your PDF output file.


In [2]:
#For package verification
#pip install yfinance
#7 stocks picked are MSFT, AAPL, GOOG, AMD, BRK.A, JPM, and DAL
#I have done a lot of repetitive download to make sure my timeline is strictly correct
#This part was completed thanks to the help of following documentations:
#https://algotrading101.com/learn/yfinance-guide/
#https://docs.python.org/3/library/datetime.html
#https://stackoverflow.com/questions/2394235/detecting-a-us-holiday
#https://numpy.org/doc/stable/reference/generated/numpy.cov.html
#https://fiscaldata.treasury.gov/api-documentation/#list-of-endpoints
#Average Interest Rates on U.S. Treasury Securities	Average Interest Rates on U.S. Treasury Securities
#/v2/accounting/od/avg_interest_rates
#Average interest rates for marketable and non-marketable securities.
#https://stackoverflow.com/questions/46741423/add-months-to-a-date-in-pandas



import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
tickers = ['AMD','MSFT','GOOG','AAPL','JPM','DAL','BLK']
benchmarks = ['SPY','IWM','DIA']
data = yf.download(tickers+benchmarks,period='10y')['Adj Close']
data['portfolio'] = data[tickers].mean(axis=1)
returns = data.pct_change()
rf = yf.download('^TNX',period = '10y')['Adj Close'].mean()

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
#Create some useful function for calculation needs
def daily_return(stock,period):
    #Return a list of daily returns of a certain stock
    prices = yf.download(stock,period = period)
    return prices['Adj Close'].pct_change()

def volatility(stock, period):
    #Calculate market volatility based on the start and end date
    re = daily_return(stock,period)
    sd_return = np.std(re)
    return sd_return*np.sqrt(len(re))

def calculate_beta(stock,benchmark,period):
    #calculate covariance using stock and index info
    stock_return = daily_return(stock,period)[1:]
    index_return = daily_return(benchmark,period)[1:]
    #I couldn't figure out why the covariance function does not work for the life of me, so I took it to my hands
    i = np.array(index_return)
    s = np.array(stock_return)
    ai = np.mean(i)
    sa = np.mean(s)
    i1 = i-ai #benchmark daily return - average return
    s1 = s-sa #stock daily return - average return 
    r = np.dot(i1,s1) #take the dot product of them to calculate covariance
    cov = r/len(i) #divide by n - 1
    sd_i = np.std(i)**2 #get the index variance
    #Ideally Covariance should form something like this from pythonm, but np.cov doesn't seem to work for this one
    #           stock    benchmark
    # stock    stock_var  cov
    #benchmark cov      benchmark_var
    #and beta is calculated using covariance of stock and benchmark/variance of stock
    return cov/sd_i

def total_return(stock,period):
    #Calculate total return over a certian period
    prices = yf.download(stock,period = period)
    return (prices['Adj Close'][-1] - prices['Adj Close'][0])/prices['Adj Close'][0]
    
def drawdown(stock, period, mode):
    prices = daily_return(stock,period)
    drawdowns = prices.rolling(5).max() - prices.rolling(5).min()
    if mode == 'a':
        return drawdowns.mean()
    elif mode == 'm':
        return drawdowns.max()
    return np.nan()

def get_price(stock):
    return yf.download(stock,period='1mo')['Adj Close'][-1]

In [6]:
#Initiate analysis by taking into ticker info over 10 years
amd = yf.download('AMD',period='10y')['Adj Close']
msft = yf.download('MSFT',period='10y')['Adj Close']
goog= yf.download('GOOG',period='10y')['Adj Close']
blk= yf.download('BLK',period='10y')['Adj Close']
aapl = yf.download('AAPL',period='10y')['Adj Close']
jpm= yf.download('JPM',period='10y')['Adj Close']
dal= yf.download('DAL',period='10y')['Adj Close']

#Create a master table
tickers = ['MSFT', 'AAPL', 'GOOG', 'AMD', 'BLK', 'JPM', 'DAL']
master_table = pd.DataFrame({'MSFT': msft,'AAPL':aapl,'GOOG':goog,'AMD':amd,'BLK':blk,'JPM':jpm,'DAL':dal})

#Create table 1 for task 1
risk_analysis = pd.DataFrame(columns=['Portfolio Weight','Annualized Volatility(Trailing 3-month)',
                                      'Beta vs SPY','Beta vs IWM', 'Beta vs DIA', 
                                      'Average Weekly Drawdown','Maximum Weekly Drawdown',
                                      'Total Return over 10 years (in %)','Annualized Total Return over 10 years (in %)'],
                             index = tickers)

#Entering weight
for x in range(len(risk_analysis['Portfolio Weight'])):
    risk_analysis['Portfolio Weight'][x] = 1/len(risk_analysis['Portfolio Weight'])

#Trailing 3 month annualized volatility
for ticker in tickers:
    risk_analysis['Annualized Volatility(Trailing 3-month)'][ticker] = volatility(ticker,'3mo')
    risk_analysis['Beta vs SPY'][ticker] = calculate_beta(ticker,'SPY','1y') 
    risk_analysis['Beta vs IWM'][ticker] = calculate_beta(ticker,'IWM','1y') 
    risk_analysis['Beta vs DIA'][ticker] = calculate_beta(ticker,'DIA','1y')
    risk_analysis['Average Weekly Drawdown'][ticker] = drawdown(ticker, '1y','a')
    risk_analysis['Maximum Weekly Drawdown'][ticker] = drawdown(ticker, '1y','m')
    x = total_return(ticker,'10y')
    risk_analysis['Total Return over 10 years (in %)'][ticker] = x*100
    risk_analysis['Annualized Total Return over 10 years (in %)'][ticker] = ((1+x)**(1/10)- 1)*100

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
#Make the table for ETFs
market_table = pd.DataFrame(columns = ['Corr', 'Cov', 'Tracking Errors', 
                                       'Sharpe Ratio','Volatility'], 
                            index=benchmarks)

market_table['Corr'] = returns[benchmarks+['portfolio']].corr().portfolio
market_table['Cov'] = returns[benchmarks+['portfolio']].cov().portfolio
for index in benchmarks:
    market_table.loc[index,'Tracking Errors'] = (returns[index]- returns['portfolio']).std()
    index_return = (data[index][-1]-data[index][0])/data[index][0] / np.sqrt(2520)
    excess_return = index_return - rf/100
    #sd isn't affected by the value of risk free, just the spread.
    sd_of_return = returns[index].std()
    market_table.loc[index,'Sharpe Ratio'] = excess_return/sd_of_return
market_table['Volatility'] = returns[benchmarks+['portfolio']][-252:].std()**(1/252)

,Portfolio Weight,Annualized Volatility(Trailing 3-month),Beta vs SPY,Beta vs IWM,Beta vs DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return over 10 years (in %),Annualized Total Return over 10 years (in %)
MSFT,0.142857,0.169367,1.24031,0.797578,1.316714,0.043633,0.093626,1024.869132,27.382627
AAPL,0.142857,0.183411,1.264288,0.858434,1.384636,0.043686,0.096725,766.55684,24.102264
GOOG,0.142857,0.205397,1.316081,0.88438,1.394451,0.048477,0.135457,631.368138,22.015107
AMD,0.142857,0.302636,2.1145,1.657461,2.174798,0.083147,0.186769,2052.288393,35.922091
BLK,0.142857,0.172235,1.183459,0.863438,1.412339,0.042108,0.101379,408.835966,17.66784
JPM,0.142857,0.149617,0.861251,0.640804,1.174778,0.037989,0.080772,311.030067,15.182728
DAL,0.142857,0.255977,1.382993,1.257041,1.74443,0.065334,0.177079,307.129855,15.072963


In [83]:
data.corr()

,AAPL,AMD,BLK,DAL,DIA,GOOG,IWM,JPM,MSFT,SPY,portfolio
AAPL,1.000000,0.980829,0.941918,0.188232,0.927867,0.969430,0.894286,0.875901,0.984586,0.962374,0.970163
AMD,0.980829,1.000000,0.928684,0.138212,0.906426,0.956114,0.871851,0.862656,0.979928,0.943717,0.959285
BLK,0.941918,0.928684,1.000000,0.370633,0.960052,0.974719,0.968222,0.948989,0.956099,0.977211,0.991404
DAL,0.188232,0.138212,0.370633,1.000000,0.480773,0.312177,0.510714,0.536613,0.250704,0.412269,0.347879
DIA,0.927867,0.906426,0.960052,0.480773,1.000000,0.960923,0.968279,0.981407,0.958459,0.991136,0.975456
GOOG,0.969430,0.956114,0.974719,0.312177,0.960923,1.000000,0.944492,0.936974,0.981797,0.984291,0.990016
IWM,0.894286,0.871851,0.968222,0.510714,0.968279,0.944492,1.000000,0.973638,0.913796,0.966510,0.962287
JPM,0.875901,0.862656,0.948989,0.536613,0.981407,0.936974,0.973638,1.000000,0.918067,0.964939,0.954193
MSFT,0.984586,0.979928,0.956099,0.250704,0.958459,0.981797,0.913796,0.918067,1.000000,0.980654,0.984273
SPY,0.962374,0.943717,0.977211,0.412269,0.991136,0.984291,0.966510,0.964939,0.980654,1.000000,0.993061


In [84]:
risk_analysis

,Portfolio Weight,Annualized Volatility(Trailing 3-month),Beta vs SPY,Beta vs IWM,Beta vs DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return over 10 years (in %),Annualized Total Return over 10 years (in %)
MSFT,0.142857,0.169367,1.24031,0.797578,1.316714,0.043633,0.093626,1024.869132,27.382627
AAPL,0.142857,0.183411,1.264288,0.858434,1.384636,0.043686,0.096725,766.55684,24.102264
GOOG,0.142857,0.205397,1.316081,0.88438,1.394451,0.048477,0.135457,631.368138,22.015107
AMD,0.142857,0.302636,2.1145,1.657461,2.174798,0.083147,0.186769,2052.288393,35.922091
BLK,0.142857,0.172235,1.183459,0.863438,1.412339,0.042108,0.101379,408.835966,17.66784
JPM,0.142857,0.149617,0.861251,0.640804,1.174778,0.037989,0.080772,311.030067,15.182728
DAL,0.142857,0.255977,1.382993,1.257041,1.74443,0.065334,0.177079,307.129855,15.072963


In [85]:
market_table

,Corr,Cov,Tracking Errors,Sharpe Ratio,Volatility,Sharpe
SPY,0.922318,0.000141,0.006193,2.79614,0.982882,2.796140
IWM,0.815928,0.000157,0.008484,1.000402,0.983719,1.000402
DIA,0.886625,0.000137,0.006994,1.96572,0.982145,1.965720


,Corr,Cov,Tracking Errors,Sharpe Ratio,Volatility,Sharpe
SPY,0.922318,0.000141,0.006193,2.79614,0.982882,2.796140
IWM,0.815928,0.000157,0.008484,1.000402,0.983719,1.000402
DIA,0.886625,0.000137,0.006994,1.96572,0.982145,1.965720


,AAPL,AMD,BLK,DAL,DIA,GOOG,IWM,JPM,MSFT,SPY,portfolio
Date,,,,,,,,,,,
2012-08-09,19.009813,4.370000,135.778564,8.056398,105.922028,15.998783,70.020378,28.017424,24.902903,116.349213,33.733412
2012-08-10,19.039516,4.340000,135.483383,8.047517,106.211845,15.990065,69.775940,28.055372,24.837582,116.539551,33.684776
2012-08-13,19.293697,4.260000,135.902863,8.225163,105.962280,16.438635,69.662422,28.070543,24.813082,116.481590,33.857712
2012-08-14,19.345457,4.120000,134.388077,8.242931,105.994476,16.654076,69.496544,28.154016,24.763771,116.498146,33.666904
2012-08-15,19.319118,4.190000,136.741806,8.225163,106.042770,16.626181,70.107697,28.131247,24.821308,116.630547,34.007832
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-03,165.899567,98.089996,690.539978,33.250000,328.160004,118.779999,189.750000,113.610001,282.470001,414.450012,214.662792
2022-08-04,165.580002,103.910004,696.010010,33.330002,327.359985,118.870003,189.350006,112.360001,283.649994,414.170013,216.244288
2022-08-05,165.350006,102.309998,695.929993,32.939999,327.970001,118.220001,190.800003,115.760002,282.910004,413.470001,216.202857


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-07-21,3.077,3.081,2.91,2.91,2.91,0


{'data': [{'record_date': '2001-01-31',
   'security_type_desc': 'Marketable',
   'security_desc': 'Treasury Notes',
   'avg_interest_rate_amt': '6.096',
   'src_line_nbr': '2',
   'record_fiscal_year': '2001',
   'record_fiscal_quarter': '2',
   'record_calendar_year': '2001',
   'record_calendar_quarter': '1',
   'record_calendar_month': '01',
   'record_calendar_day': '31'},
  {'record_date': '2001-01-31',
   'security_type_desc': 'Marketable',
   'security_desc': 'Treasury Bonds',
   'avg_interest_rate_amt': '8.450',
   'src_line_nbr': '3',
   'record_fiscal_year': '2001',
   'record_fiscal_quarter': '2',
   'record_calendar_year': '2001',
   'record_calendar_quarter': '1',
   'record_calendar_month': '01',
   'record_calendar_day': '31'},
  {'record_date': '2001-01-31',
   'security_type_desc': 'Marketable',
   'security_desc': 'Treasury Inflation-Indexed Notes',
   'avg_interest_rate_amt': '3.772',
   'src_line_nbr': '4',
   'record_fiscal_year': '2001',
   'record_fiscal_quarter

1.8872586057514427

1.8872586057514424